In [12]:
import re
import csv
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import groupby
from collections import namedtuple
from pandas.io.json import json_normalize


import nltk
import pymorphy2
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize


from gensim.models import Doc2Vec
from gensim.models import Word2Vec
from gensim.models import KeyedVectors


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.semi_supervised import LabelPropagation
from sklearn.semi_supervised import LabelSpreading
from sklearn.cluster import KMeans
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

In [13]:
def cleaning(text):
    text = re.sub(r'[^\w\s]','',text)
    tokenized = []
    words = word_tokenize(text)
    for word in words:
        p = morph.parse(word)[0]
        tokenized.append(p.normal_form)
    tokenized = [token for token in tokenized if token not in stopWords\
                and token != " " \
                and token != "—" \
                and token != "«" \
                and token != "»" \
                and token != ".." \
                and token.strip() not in punctuation]
    return tokenized

In [14]:
def shaping (rev):
    shapeddata = []
    analyzedDocument = namedtuple('AnalyzedDocument', 'words tags')
    for i, text in enumerate(rev):
        tags = [i]
        shapeddata.append(analyzedDocument(text, tags))
    return shapeddata

In [15]:
file = 'classic_poems.json'
with open(file, 'r', encoding='utf-8') as fh: #открываем файл на чтение
    data = json.load(fh)
morph = pymorphy2.MorphAnalyzer()
stopWords = stopwords.words('russian')

In [16]:
poems = pd.DataFrame(data)
blok = poems[lambda x: x['poet_id'] == 'blok']
cleaned_blok = blok['content'].apply(cleaning)
dfpoems = cleaned_blok
Blokpoems = dfpoems.tolist()

In [18]:
labels = [0,1,2,1,2,1,1,0,1,1,0,1,1,2,0,2,1,1,1,1,1,1,1,1,1,1,2,2,2,0,0,0,1,1,2,2,1,1,1,2,2,0,1,1,1,0,1,0,0,2,2, 2, 1, 2, 0, 0, 2, 1, 0, 0, 1, 1, 0 ,0 ,1, 2, 1, 0, 2, 2, 0, 0, 0, 1, 1, 1, 0, 1, 2, 1, 0, 1, 0, 0, 1, 0, 1, 0, 2, 2, 1, 0, 2, 1, 0, 1, 1, 1]

In [19]:
alllabels = []
for el in labels:
    alllabels.append(el)
for i in range (0, 979):
    alllabels.append(-1)

In [20]:
shaped_poems = shaping(Blokpoems)

In [21]:
dv = Doc2Vec(shaped_poems, vector_size = 100, window = 300, min_count = 1, workers = 4)
docvecs = []
for i in range(0,1077):
    docvecs.append(dv[i])



In [22]:
labeled_spr = LabelSpreading(kernel='knn')
labelsenttest = labeled_spr.fit(docvecs, alllabels)
labelstest = labelsenttest.transduction_

C:\Users\1255971\Anaconda3\lib\site-packages\sklearn\semi_supervised\label_propagation.py:297: RuntimeWarning: invalid value encountered in true_divide
  self.label_distributions_ /= normalizer
